## Create Dataset for filtering: Idea Startup

In [17]:
pip install requests Pillow beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [18]:
import requests, os
from bs4 import BeautifulSoup

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

response = requests.get("https://ideasai.com/data-startup-ideas")
soup = BeautifulSoup(response.text, "html.parser")

ideas = soup.findAll('h3', class_='idea')
idea_descriptions = [desc.text for desc in ideas]
print(idea_descriptions, len(idea_descriptions))

votes = soup.findAll('span', class_='votes')
voting = [v.text for v in votes]
print(voting)

print (len(voting), len(idea_descriptions))

import csv

file_path='data/valid_ideas.csv'
print(os.getcwd())

  # Try to open the file for reading
with open(file_path, "x+") as file:
  # Read the contents of the file
  for idx in range(len(voting)):
    desc = idea_descriptions[idx].replace('"', '')
    vote = voting[idx].replace(',', '')
    sen = ';'.join([desc, '1', vote, '\n'])
    file.write(sen)

['A platform for buying, selling, trading, and renting bikes. The company’s API provides access to data on bikes, stores, and repairs, and it currently has 30,000 bike listings.', 'A data-driven tool that helps companies build better websites by looking at the user journeys they’re taking across the internet.', 'A company that helps users track their data with a “health score” that shows how healthy they are.', 'A data science company that helps people better understand their personal data.', 'A device that can measure how much a plant needs to water its roots and send that data to a mobile app.', 'This startup has built a medical app to help in-home care providers manage their client’s data.', 'A free library of “crowd sourced” electronics repair instructions, with a database of more than 100,000 step-by-step guides for common problems with over a million unique parts.', 'A cloud-based software platform for the professional education market. The startup is building a graph database fo

FileExistsError: [Errno 17] File exists: 'data/valid_ideas.csv'

In [ ]:
import requests, json
from bs4 import BeautifulSoup

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

response = requests.get("https://randomwordgenerator.com/json/sentences.json")
type(response.text)
data = json.loads(response.text)
print(data["data"])
sentences = data["data"]

import csv

with open('data/invalid-sentences.csv', 'x+', newline='') as file:
  writer = csv.writer(file)
  
  for idx in range(len(sentences)):
    writer.writerow([sentences[idx]["sentence"], 0])
    

[{'sentence': 'The quick brown fox jumps over the lazy dog.'}, {'sentence': 'My Mum tries to be cool by saying that she likes all the same things that I do.'}, {'sentence': 'A purple pig and a green donkey flew a kite in the middle of the night and ended up sunburnt.'}, {'sentence': 'Last Friday I saw a spotted striped blue worm shake hands with a legless lizard.'}, {'sentence': 'A song can make or ruin a person’s day if they let it get to them.'}, {'sentence': 'Sometimes it is better to just walk away from things and go back to them later when you’re in a better frame of mind.'}, {'sentence': 'Writing a list of random sentences is harder than I initially thought it would be.'}, {'sentence': 'Lets all be unique together until we realise we are all the same.'}, {'sentence': 'If I don’t like something, I’ll stay away from it.'}, {'sentence': 'I love eating toasted cheese and tuna sandwiches.'}, {'sentence': 'If you like tuna and tomato sauce, try combining the two, it’s really not as bad

In [ ]:
import requests, json
from bs4 import BeautifulSoup

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

response = requests.get("https://www.randomsentencegen.com/sentence-with-startup")
soup = BeautifulSoup(response.text, "html.parser")

ideas = soup.findAll('tr', class_='item')
idea_descriptions = [desc.text for desc in ideas]
print(idea_descriptions, len(idea_descriptions))

import csv

with open('data/invalid-sentences.csv', 'x+', newline='') as file:
  writer = csv.writer(file)
  
  for idx in range(len(sentences)):
    writer.writerow([sentences[idx]["sentence"], 0])
    

## Filtering by LSTM

### Import

In [34]:
import pandas as pd

invalid_0 = pd.read_csv("data/invalid_sentences.csv", delimiter=';')
invalid_1 = pd.read_csv("data/random_str.csv", delimiter=',')
semantic = pd.read_csv("data/valid_ideas.csv", delimiter=';', usecols=['sentence', 'spam'])

invalid_0.head()
invalid_1.head()
semantic.head()

frames = [invalid_0, semantic, invalid_1]
 
df = pd.concat(frames)

df.head()

sentence  spam
0       The quick brown fox jumps over the lazy dog.     0
1  My Mum tries to be cool by saying that she lik...     0
2  A purple pig and a green donkey flew a kite in...     0
3  Last Friday I saw a spotted striped blue worm ...     0
4  A song can make or ruin a person’s day if they...     0

### Sequence Model

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -U tensorflow-text

In [ ]:
!pip install tf-models-official

In [ ]:
!pip install matplotlib

In [20]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

2023-10-09 01:05:02.652502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
!pip install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post9-py3-none-any.whl size=2952 sha256=ed1b6450938f25fc71463c93c850f9d3acf3e5e3f7a63bffdc986b9fec4e877f
  Stored in directory: /Users/nghinguyen/Library/Caches/pip/wheels/ef/63/d1/f1671e1e93b7ef4d35df483f9b2485e6dd21941da9a92296fb
Successfully built sklearn


In [48]:
from sklearn.model_selection import train_test_split
import numpy as py

#Create test and train samples from one dataframe with pandas?
train_data, test_data = train_test_split(df, test_size=0.3)

Build Vocabulary - TextVectorization

In [ ]:
from tensorflow.keras.layers import TextVectorization


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


# Now that the vectorize_layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
# Let's call `adapt`:
train_without_label = train_data.loc[:, df.columns == 'sentence']
train_dataset = tf.data.Dataset.from_tensor_slices(train_without_label.to_dict(orient="list"))
list_data = train_without_label.values.tolist()

# Adapt with Dataframe
vectorize_layer.adapt(train_without_label, batch_size=64)

# Adapt with list
vectorize_layer.adapt(list_data, batch_size=64)
vectorize_layer.get_vocabulary()
